# Preprocessing, Data Visualization, EDA

## Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pydicom
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
from matplotlib import cm
import os
import seaborn as sns
from skimage import measure
import keras
import tensorflow as tf
import cv2
%matplotlib inline

#setting up for customized printing
from IPython.display import Markdown, display
from IPython.display import HTML
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [ ]:
def random_color():
    rgbl=[1,0,0]
    np.random.shuffle(rgbl)
    return tuple(rgbl)  

def distplot(data, figRows,figCols,xSize, ySize, features, colors):
    f, axes = plt.subplots(figRows, figCols, figsize=(xSize, ySize))
    
    features = np.array(features).reshape(figRows, figCols)
    colors = np.array(colors).reshape(figRows, figCols)
    
    for row in range(figRows):
        for col in range(figCols):
            if (figRows == 1 and figCols == 1) :
                axesplt = axes
            elif (figRows == 1 and figCols > 1) :
                axesplt = axes[col]
            elif (figRows > 1 and figCols == 1) :
                axesplt = axes[row]
            else:
                axesplt = axes[row][col]
            plot = sns.distplot(data[features[row][col]], color=colors[row][col], ax=axesplt, kde=True, hist_kws={"edgecolor":"k"})
            plot.set_xlabel(features[row][col],fontsize=20)
            
def boxplot(data, figRows,figCols,xSize, ySize, features, colors, hue=None, orient='h'):
    f, axes = plt.subplots(figRows, figCols, figsize=(xSize, ySize))
    
    features = np.array(features).reshape(figRows, figCols)
    colors = np.array(colors).reshape(figRows, figCols)
    
    for row in range(figRows):
        for col in range(figCols):
            if (figRows == 1 and figCols == 1) :
                axesplt = axes
            elif (figRows == 1 and figCols > 1) :
                axesplt = axes[col]
            elif (figRows > 1 and figCols == 1) :
                axesplt = axes[row]
            else:
                axesplt = axes[row][col]
            plot = sns.boxplot(features[row][col], data= data, color=colors[row][col], ax=axesplt, orient=orient, hue=hue)
            plot.set_xlabel(features[row][col],fontsize=20)
            
def countplot(data, figRows,figCols,xSize, ySize, features, colors=None,palette=None,hue=None, orient=None, rotation=90):
    f, axes = plt.subplots(figRows, figCols, figsize=(xSize, ySize))
    
    features = np.array(features).reshape(figRows, figCols)
    if(colors is not None):
        colors = np.array(colors).reshape(figRows, figCols)
    if(palette is not None):
        palette = np.array(palette).reshape(figRows, figCols)
    
    for row in range(figRows):
        for col in range(figCols):
            if (figRows == 1 and figCols == 1) :
                axesplt = axes
            elif (figRows == 1 and figCols > 1) :
                axesplt = axes[col]
            elif (figRows > 1 and figCols == 1) :
                axesplt = axes[row]
            else:
                axesplt = axes[row][col]
                
            if(colors is None):
                plot = sns.countplot(features[row][col], data=data, palette=palette[row][col], ax=axesplt, orient=orient, hue=hue)
            elif(palette is None):
                plot = sns.countplot(features[row][col], data=data, color=colors[row][col], ax=axesplt, orient=orient, hue=hue)
            plot.set_title(features[row][col],fontsize=20)
            plot.set_xlabel(None)
            plot.set_xticklabels(rotation=rotation, labels=plot.get_xticklabels(),fontweight='demibold',fontsize='large')
            
def point_box_bar_plot(data, row, col, target, figRow, figCol, palette='rocket', fontsize='large', fontweight='demibold'):
    sns.set(style="whitegrid")
    f, axes = plt.subplots(3, 1, figsize=(figRow, figCol))
    pplot=sns.pointplot(row,col, data=data, ax=axes[0], linestyles=['--'])
    pplot.set_xlabel(None)
    pplot.set_xticklabels(labels=pplot.get_xticklabels(),fontweight=fontweight,fontsize=fontsize)
    bxplot=sns.boxplot(row,col, data=data, hue=target, ax=axes[1],palette='viridis')
    bxplot.set_xlabel(None)
    bxplot.set_xticklabels(labels=bxplot.get_xticklabels(),fontweight=fontweight,fontsize=fontsize)
    bplot=sns.barplot(row,col, data=data, hue=target, ax=axes[2],palette=palette)
    bplot.set_xlabel(row,fontsize=20)
    bplot.set_xticklabels(labels=bplot.get_xticklabels(),fontweight=fontweight,fontsize=fontsize)            
def catdist(data, cols):
    dfs = []
    for col in cols:
        colData = pd.DataFrame(data[col].value_counts(), columns=[col])
        colData['%'] = round((colData[col]/colData[col].sum())*100,2)
        dfs.append(colData)
        display(colData)

## Setup Data

In [ ]:
def formatMetadataString(val):
    return str(val).split(':')[1].replace('\'', '')

dataDir = '/kaggle/input/rsna-pneumonia-detection-challenge'
trainDataDir = 'stage_2_train_images'
testDataDir = 'stage_2_test_images'
trainfiles = glob.glob(os.path.join(dataDir, trainDataDir, "*.dcm"))

printmd('**Reading the metadata from dicom training files**', color='brown')

image_data = []
for f in trainfiles:    
    ds = pydicom.dcmread(f, stop_before_pixels=True)
    patientId = formatMetadataString(ds['PatientID'])
    age = formatMetadataString(ds['PatientAge'])
    gender = formatMetadataString(ds['PatientSex'])
    viewPos = formatMetadataString(ds['ViewPosition'])    
    image_data.append([patientId, int(age), gender, viewPos])
    
image_metadata_df = pd.DataFrame(image_data, columns=['patientId', 'Age', 'Gender','ViewPosition'])        
image_metadata_df.head(3)

In [ ]:
print('Storing the DICOM files as a dictionary under trainDicomFiles such that the keys are the patient\
ids and the values are the path of the files corresponding to the image.')
trainDicomFiles = {}
testDicomFiles = {}

for root, dirs, files in os.walk(os.path.join(dataDir, trainDataDir)):
    for fileName in files:
        if '.dcm' in fileName.lower():
            trainDicomFiles[fileName.split(sep='.')[0]] = os.path.join(dataDir, trainDataDir, fileName)     
        
print('\nGenerating the data frame for the training data labels.')
trainLabelDf = pd.read_csv(os.path.join(dataDir, 'stage_2_train_labels.csv'))
trainLabelDetailedDf = pd.read_csv(os.path.join(dataDir, 'stage_2_detailed_class_info.csv'))

# Joining the class information from the detailed class info CSV to the train labels CSV.
trainLabelDf['class'] = trainLabelDetailedDf['class']

# As the training labels file contain multiple rows for each patient corresponding to the number of bounding boxes present
# in case of a person with lung opacity, we can group the labels by patient id to find number of unique patients in 
# the label file.

print('\nGenerating the dictionary for test DICOM files under testDicomFiles.')
for root, dirs, files in os.walk(os.path.join(dataDir, testDataDir)):
    for fileName in files:
        if '.dcm' in fileName.lower():
            testDicomFiles[fileName.split(sep='.')[0]] = os.path.join(dataDir, testDataDir, fileName)

print('Total test data files: ', len(testDicomFiles))

#print('\nBelow is a sample of the content of training label data frame.')
#display(trainLabelDf.head(10))

print('\nAs the training label data frame may contain multiple records for a patient to represent the bounding box, we can \
store the bounding box labels in a property called boundingBox that contains a array of box data stored as \
{x, y, width, height}')

uniquePatientIds = np.unique(trainLabelDf.patientId)
trainLabelAndBoundingBoxDf = pd.DataFrame(columns=['patientId', 'target', 'class', 'boundingBox', 'num_of_opacities']
                                          , index=uniquePatientIds)

patientIdBasedGroups = trainLabelDf.groupby('patientId')
#counter = 0
for patientId, group in patientIdBasedGroups:
#     if counter > 20:
#         break
    trainLabelAndBoundingBoxDf.loc[patientId]['patientId'] = patientId
    trainLabelAndBoundingBoxDf.loc[patientId]['num_of_opacities'] = 0
    boundingBox = []
    for index, groupItem in group.iterrows():
        trainLabelAndBoundingBoxDf.loc[patientId]['target'] = groupItem['Target']
        trainLabelAndBoundingBoxDf.loc[patientId]['class'] = groupItem['class']
        
        if(not np.isnan(groupItem['x'])):
            boundingBox.append((groupItem['x'], groupItem['y'], groupItem['width'], groupItem['height']))
            trainLabelAndBoundingBoxDf.loc[patientId]['num_of_opacities'] += 1
            
    trainLabelAndBoundingBoxDf.loc[patientId]['boundingBox'] = boundingBox
#     counter += 1

display(trainLabelAndBoundingBoxDf.head())

print('\nShape of training labels with bounding box list: ', trainLabelAndBoundingBoxDf.shape)

In [ ]:
printmd('**Combining training data with labels and bounding boxes with the image metadata containing age, gender and view position attributes**', color='brown')
trainLabelAndBoundingBoxDf_2 = trainLabelAndBoundingBoxDf.reset_index().drop('index', axis=1)
patient_df = image_metadata_df.join(trainLabelAndBoundingBoxDf_2, how='inner', rsuffix='_2')
patient_df.drop('patientId_2', axis=1, inplace=True)
patient_df.head()

### Details of the Attributes

1. **patientId**: Unique Identifier for a Patient.<br>
2. **Age** : Patient's age in completed years.<br>
3. **Gender**: Patient's gender is a categorical attribute with values of 'M'(male) & 'F'(female)<br>
4. **ViewPosition**: This is a categorial attribute with values of 'PA' and 'AP'<br>
    a. **PA - PosteroAnterior**: From back to front: When a chest x-ray is taken with the front against the film plate and the x-ray machine in back of the patient it is called an posteroanterior (PA) view.<br>
    b. **AP - Anteroposterior**: From front to back. When a chest x-ray is taken with the back against the film plate and the x-ray machine in front of the patient it is called an anteroposterior (AP) view. <br>
5. **class**: Categorical attribute with the following values,<br>
    a. **Lung Opacity**<br>
    b. **Normal**<br>
    c. **No Lung Opacity / Not Normal**<br>
6. **boundingBox** - These values denotes position of the lung opacity in the CXR image. There can be multiple opacities for a person<br>
7. **target** - Target attribute denotes whether a patient is Pneumonia positive(1) or negative(0)<br>
8. **num_of_opacities** - This denotes the number of opacities diagnosed for a person. 

## Perform Basic EDA

In [ ]:
print('Total training data files: ', len(trainDicomFiles))
print('Total rows in the labels file: ', trainLabelDf.shape[0])
print('Total unique patients in the labels file: ', trainLabelDf.groupby('patientId').count().shape[0])
print('Unique target values are:', np.unique(trainLabelDf['Target']).tolist())
print('Unique class values are: ', np.unique(trainLabelDf['class']).tolist())

In [ ]:
printmd('**We can check if the bounding boxes are present only for the patients with lung opacity.**', color='brown')
recordsWithLungOpacityAndBoundingBox = trainLabelAndBoundingBoxDf[(trainLabelAndBoundingBoxDf['boundingBox'].apply(lambda row: len(row) > 0)) & (trainLabelAndBoundingBoxDf['class'] == 'Lung Opacity')].shape[0]
recordsWithNoLungOpacityAndBoundingBox = trainLabelAndBoundingBoxDf[(trainLabelAndBoundingBoxDf['boundingBox'].apply(lambda row: len(row) > 0)) & (trainLabelAndBoundingBoxDf['class'] != 'Lung Opacity')].shape[0]
print('Record count with bounding box for patients with lung opacity: ', recordsWithLungOpacityAndBoundingBox)
print('Record count with bounding box for patients with no lung opacity: ', recordsWithNoLungOpacityAndBoundingBox)
print('We can see that bounding boxes are present for only patients with a lung opacity i.e. the patients having pneumonia.')

## Analysis of DICOM file samples

In [ ]:
printmd('**Analyzing a sample with normal lung image.**', color='brown')
sampleNormalLungPatient = trainLabelAndBoundingBoxDf[trainLabelAndBoundingBoxDf['class'] == 'Normal'].iloc[0]
sampleNormalLungDicomFile = pydicom.read_file(trainDicomFiles[sampleNormalLungPatient.patientId])
#display(sampleNormalLungDicomFile)

printmd('**Below is the image plot for the normal lung image stored in the DICOM file.**', color='brown')
plt.imshow(sampleNormalLungDicomFile.pixel_array, cmap=cm.bone)
plt.show()

printmd('**Below is the class target for the normal lung sample.**', color='brown')
sampleNormalLungPatient

In [ ]:
printmd('**Analyzing a sample with lung opacity image.**', color='brown')
sampleLungOpacityPatient = trainLabelAndBoundingBoxDf[trainLabelAndBoundingBoxDf['class'] == 'Lung Opacity'].iloc[0]
sampleLungOpactiyDicomFile = pydicom.read_file(trainDicomFiles[sampleLungOpacityPatient.patientId])
#display(sampleLungOpactiyDicomFile)

printmd('**Below is the image plot for the lung opacity image stored in the DICOM file.**', color='brown')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(sampleLungOpactiyDicomFile.pixel_array, cmap=cm.bone)
for box in sampleLungOpacityPatient['boundingBox']:
    #print(box)
    ax.add_patch(Rectangle((box[0], box[1]), box[2], box[3], linewidth=1, edgecolor=random_color(), facecolor='none'))
plt.show()

printmd('**Below is the class target for the lung opacity sample.**', color='brown')
sampleLungOpacityPatient

In [ ]:
printmd('**Analyzing a sample with no lung opacity and not normal image.**', color='brown')
sampleLungNotNormalPatient = trainLabelAndBoundingBoxDf[trainLabelAndBoundingBoxDf['class'] == 'No Lung Opacity / Not Normal'].iloc[0]
sampleLungNotNormalDicomFile = pydicom.read_file(trainDicomFiles[sampleLungNotNormalPatient.patientId])
#display(sampleLungNotNormalDicomFile)

printmd('**Below is the image plot for the no lung opacity and not normal image stored in the DICOM file.**', color='brown')
plt.imshow(sampleLungNotNormalDicomFile.pixel_array, cmap=cm.bone)
plt.show()

printmd('**Below is the class target for the no lung opacity and not normal image sample.**', color='brown')
sampleLungNotNormalPatient

### Dealing with missing values

In [ ]:
print('We can check the missing values in the train class files.')

print('Using the count function on the labels data frame, we can find if there exist any column with "nan" values.')
display(trainLabelAndBoundingBoxDf.count())
print('\nAs we can see that all patientId, target and class does not contain any nan values as count is same as total number \
of patients.')

print('\nWe can check the unique target and class values to determine if there is any ambiguous/missing data.')
print('Unique target values are:', np.unique(trainLabelAndBoundingBoxDf['target']).tolist())
print('Unique class values are: ', np.unique(trainLabelAndBoundingBoxDf['class']).tolist())
print('We can see there exist no invalid value for target and class data.')

print('\nNow we can analyze the box coordinates for all the data points where Target is 1 (Pneumonia detected).')
display(trainLabelDf[trainLabelDf['Target'] == 1].describe())

print('\nWe can see from statistics all the x, y coordinates contain valid values and none of x, y, width and height columns\
 contain the min value as 0 indicating the values would be correctly recorded.')

print('\nWe can check to see if all the patients within the train label data set has a corresponding DICOM image available.')
for labelRow in trainLabelAndBoundingBoxDf.iterrows():
    if labelRow[1].patientId not in trainDicomFiles.keys():
        print(f'{labelRow[1].patientId} is not present in the DICOM files.')

print('We can see that all the patients in labels file have a corresponding DICOM image file.')

## Distributions of Numeric Attribute - Age

In [ ]:
distplot(patient_df, 1,1, 12,5, 
         ['Age'], 
         ['red'])

### Observations

There is a good distribution of people from all ages in the data. From the plot, it looks is normally distributed with majority of people around ages 40 to 60. 


## Distribution of Categorical Attributes

In [ ]:
catdist(patient_df, ['Gender', 'class', 'target', 'ViewPosition'])

## Bivariate Analysis

In [ ]:
printmd('**Attributes target - Age**', color='brown')
point_box_bar_plot(patient_df, 'target', 'Age', 'target', 15, 10, palette='winter')

### Observations

1. Average age of people who are normal and who are affected by pneumonia are the same at around 47. 
2. Distribution of people's age is quite similar for both Normal and Pneumonia affected. 

In [ ]:
printmd('**Attributes class - Age**', color='brown')
point_box_bar_plot(patient_df, 'class', 'Age', 'class', 15, 10, palette='winter')

### Observations

1. Average age of people across different classes (Normal, Lung Opacity, No Lung Opacity/Not Normal) are same at around 47. 
2. Distribution of people's age is quite similar for class types. 

In [ ]:
printmd('**Distribution of data/images accross different classes.**', color='brown')

fig, axes = plt.subplots(1)
col1plot = sns.countplot(x='class', data=patient_df)
for p in col1plot.patches:
    col1plot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

fig.set_figwidth(10)
fig.set_figheight(8)
plt.show()

### Observations

There is a good ratio of class distributions. Different class types are distributed as below,<br>
1. No Lung Opacity / Not Normal - 11821 - 44.30%
2. Normal - 8851 - 33.17%
3. Lung Opacity - 6012 - 22.53%

In [ ]:
printmd('**Distribution of categorical attributes wrt target attribute**', color='brown')
countplot(patient_df, 2,2, 20,10, 
         ['target', 'Gender', 'ViewPosition', 'class'], 
         palette=['viridis', 'Dark2_r', 'winter', 'Set1'], hue='target', rotation=60)

### Observations

1. There are more people not diagnosed with Pneumonia. Target attribute is distributed as, <br>
    a. 0 - 20672 - 77.47 <br>
    b. 1 - 6012 - 22.53 <br>
2. Gender - Target -> Number of male patients is quite similar in count as the number of female patients for both Normal and Pneumonia affected, with male patients slightly higher. 
3. ViewPosition - Target -> Number of patients who have given CXR in PA viewposition is quite same in count as the number of patients in AP viewposition for both Normal and Pneumonia affected. 
4. Class - Target -> No Lung Opacity / Not Normal class is also considered as Non-Pneumonic along with Normal class. Lung Opacity confirms Positive Pneumonia in patients. 

### Observations from the visualizations

In [ ]:
printmd('**We can see the data is distributed across the different classes with adequate representation from each class. \
Thereby, we can use the existing dataset for model generation.**', color='blue')

# Model Building

### 1) Building a pneumonia detection model starting from basic CNN and then improving upon it.

In [ ]:
print('As we can have multiple bounding boxes for the lung opacity condition, we can consider a mask with all region with lung \
marked by all 1s to indicate the presence of lung opacity. This can help us represent multiple lung opacity regions in a single mask.')
print('\nAdditionally, we would be using a convolutional neural network based on resnet to receive high level of accuracy.')

In [ ]:
print('As the data available for training is large, we can build a data generator class that can help in building the input data \
in batches to avoid overwhelming the system memory.')
print('We can extend the keras sequence class to get the data generator.')

class LungDataGenerator(keras.utils.Sequence):
    def __init__(self, patientIds, batch_size=32, dim=(1024,1024), shuffle=True):
        self.patientIds = patientIds
        self.batch_size = batch_size
        self.dim=dim
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __load__(self, patientId):
        # Load the dicom file as a numpy array
        dicom_image = pydicom.dcmread(trainDicomFiles[patientId]).pixel_array

        # Create an empty mask to hold the list of bounding boxes as image segments.
        bounding_box_mask = np.zeros(dicom_image.shape) 

        # Generate the mask using the label data frame
        targetLabelDf = trainLabelAndBoundingBoxDf.loc[patientId]
        if targetLabelDf['target'] == 1:
            for box in targetLabelDf['boundingBox']:
                x, y, width, height = [int(item) for item in box]
                bounding_box_mask[y:y+height, x:x+width] = 1

        # Resizing the image for reduction (to help build faster model).
        dicom_image = cv2.resize(dicom_image, (self.dim[0], self.dim[1]))
        bounding_box_mask = cv2.resize(bounding_box_mask, (self.dim[0], self.dim[1]))
        dicom_image = np.expand_dims(dicom_image, -1)
        bounding_box_mask = np.expand_dims(bounding_box_mask, -1)
        target = targetLabelDf['target']
        return dicom_image, bounding_box_mask, target

    def __len__(self):
        # Represents the number of batches per epoch
        return int(len(self.patientIds) / self.batch_size)
    
    def __getitem__(self, index):
        # Generates the indexes associated with one batch
        targetPatientIds = self.patientIds[index * self.batch_size : (index + 1) * self.batch_size]
        
        items = [self.__load__(id) for id in targetPatientIds]

        # Zip the images and masks for the target patients.
        imgs, masks, targets = zip(*items)

        # Create numpy batch
        imgs = np.array(imgs)
        masks = np.array(masks)
        targets = np.array(targets)
        return imgs, masks, targets
        
    def on_epoch_end(self):
        # Shuffles the data after every epoch
        if self.shuffle:
            np.random.shuffle(self.patientIds)

In [ ]:
print('Now we can define the metrics for loss reduction as below:')
print('Define IOU or jaccard loss function')
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
    return 1 - score

print('Combine BCE and IOU loss')
def iou_bce_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * iou_loss(y_true, y_pred)
    #return iou_loss(y_true, y_pred)

print('Get mean IOU as a metric')
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis = [1, 2, 3])
    union = tf.reduce_sum(y_true, axis = [1, 2, 3]) + tf.reduce_sum(y_pred, axis = [1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

In [ ]:
print('We would create the convolutional neural network using resnet blocks.\
The model consists of a downsampling block (consisting of Batch normalization, leaky ReLU, Convolutional 2D and max pool layer) \
and a resnet block (consisting of batch normalization, leaky ReLU and convolutional 2D layers). The downsampling and resnet \
blocks are repeated based on configuration parameters of n_blocks and depth. In the end we upsample the data points to the same \
size as before in order to get the target mask.')
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=4, isDropoutAdded=False):
    # Input
    inputs = keras.Input(shape=(input_size, input_size, 1))

    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # Residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        
        if isDropoutAdded:
            # Add a dropout layer post down-sampling
            x = keras.layers.Dropout(0.2)(x)
        
        for b in range(n_blocks):
            x = create_resblock(channels, x)
            if isDropoutAdded:
                # Add a dropout layer post each resnet block
                x = keras.layers.Dropout(0.2)(x)

    # Output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
print('Generating the network instance.')
model = create_network(input_size=256, channels=32, n_blocks=2, depth=4, isDropoutAdded=False)
# Using the optimizer with default learning rate of 0.01.
learn_rate = 0.001
adamOpt = keras.optimizers.Adam(learning_rate=learn_rate)
model.compile(optimizer=adamOpt, loss=iou_bce_loss, metrics=['accuracy', mean_iou])

epoch_count = 5

# Define cosine annealing
def cosine_annealing(x):
    lr = learn_rate
    epochs = epoch_count
    return lr * (np.cos(np.pi * x / epochs) + 1.) / 2

model.summary()

In [ ]:
print('Training the model.')
learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

training_data_percent = 0.7
training_data_end_index = int(trainLabelAndBoundingBoxDf.shape[0] * training_data_percent)
training_patientIds_box = trainLabelAndBoundingBoxDf['patientId'][0:training_data_end_index]
validation_patientIds_box = trainLabelAndBoundingBoxDf['patientId'][training_data_end_index:]

params = {'dim': (256, 256), 'batch_size': 32, 'shuffle': True}
training_data_generator_box = LungDataGenerator(training_patientIds_box, **params)
validation_data_generator_box = LungDataGenerator(validation_patientIds_box, **params)

history = model.fit(training_data_generator_box, validation_data=validation_data_generator_box, callbacks=[learning_rate], epochs=epoch_count, workers=4, use_multiprocessing=True)

In [ ]:
print('Below is the representation of variation of loss and mean_iou for training and validation data.')
plt.figure(figsize=(15,6))
plt.subplot(121)
plt.plot(history.epoch, history.history["loss"], label="Train loss")
plt.plot(history.epoch, history.history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(122)
plt.plot(history.epoch, history.history["mean_iou"], label="Train iou")
plt.plot(history.epoch, history.history["val_mean_iou"], label="Valid iou")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print('We can generate the y_pred and y_true arrays to hold the target values (predicted and actual) for all the validation \
data points.')
def get_confusion_matrix(target_model, data_generator, isDisplayRequired=False):
    y_pred = []
    y_true = []

    for imgs, masks, targets in data_generator:
        predictions = target_model.predict(imgs)

        # Loop through the predictions along with the actual targets
        for img, mask, prediction, target in zip(imgs, masks, predictions, targets):
            # Apply threshold to the predicted mask
            comp = prediction[:,:,0] > 0.5

            # Apply connected components.
            comp = measure.label(comp)

            # Take the initial prediction as 0
            predTarget = 0

            # Apply bounding boxes
            for region in measure.regionprops(comp):
                # If a region is found mark the target value as 1
                predTarget = 1

            y_pred.append(predTarget)
            y_true.append(target)
            
    cm = confusion_matrix(y_true, y_pred)
    
    if isDisplayRequired:
        print('Below is the confusion matrix:')
        ax = sns.heatmap(cm, annot=True, fmt='d')
        plt.show()

        print('\nBelow is the classification report based on the actual and predicted target values:')
        print(classification_report(y_true, y_pred, labels=[0,1]))
        
    return cm

cm = get_confusion_matrix(model, validation_data_generator_box, True)

In [ ]:
print('We can now analyze the predictions on the validation data.')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
for imgs, masks, targets in validation_data_generator_box:
    # Predict a batch of images
    predictions = model.predict(imgs)
    # Create a plot for the images
    fig, axes = plt.subplots(3, 5, figsize=(20, 15))
    axes = axes.ravel()
    axidx = 0
    
    # Loop through the batch
    for img, mask, prediction, target in zip(imgs, masks, predictions, targets):
        axes[axidx].imshow(img[:,:,0])
        
        # Apply threshold to true mask
        comp = mask[:,:,0] > 0.5
        
        # Apply connected components.
        comp = measure.label(comp)
        
        # Apply bounding boxes
        for region in measure.regionprops(comp):
            y1, x1, y2, x2 = region.bbox
            height = y2-y1
            width = x2-x1
            axes[axidx].add_patch(patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='b', facecolor='none'))
            
        # Apply threshold to the predicted mask
        comp = prediction[:,:,0] > 0.5
        
        # Apply connected components.
        comp = measure.label(comp)
        
        predTarget = 0
        
        # Apply bounding boxes
        for region in measure.regionprops(comp):
            # If a region is found mark the target value as 1
            predTarget = 1
            y1, x1, y2, x2 = region.bbox
            height = y2-y1
            width = x2-x1
            axes[axidx].add_patch(patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='r', facecolor='none'))
        
        axes[axidx].text(0.05, 0.95, f'Actual target: {target} \nPredicted target: {predTarget}', transform=axes[axidx].transAxes, fontsize=14, verticalalignment='top', bbox=props)
        axidx += 1
        if(axidx >= 15):
            break;
        
    plt.show()
    
    # Display only one plot per batch
    break


In [ ]:
print('We have successfully built a Pneumonia Detection model using ResNet architecture that is able to predict Pneumonia along with the lung opacity region from the CXR images with an accuracy of 96%.')
print('We can improve the performance of the current model by performing hyper-parameter tuning for the model by \
\n1) Adjusting the number of ResNet blocks used for the model\
\n2) Tuning the depth for the model.\
\n3) Changing the target size to which the input images are reduced.\
\n4) Fine tuning the number of channels used throughout the model.')

print('\nAdditionally, we would look at some pre-built model libraries available for masked R-CNN like matterport to see if it \
improves the performance.')

print('\nWe would also try using a pre-trained model like mobilenet by training the last few layers of the model.')

In [ ]:
print('First, we can try the U-Net model for bounding box detection.')
def unet(input_size=(256,256,1)):
    inputs = keras.Input(input_size)
    
    conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = keras.layers.concatenate([keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = keras.layers.concatenate([keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = keras.layers.concatenate([keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = keras.layers.concatenate([keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    return keras.Model(inputs=[inputs], outputs=[conv10])

model_unet = unet(input_size=(256,256,1))
model_unet.compile(optimizer='adam', loss=iou_bce_loss, metrics=['accuracy', mean_iou])
model_unet.summary()

In [ ]:
print('Training the U-Net model.') 
learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

training_data_percent = 0.7
training_data_end_index = int(trainLabelAndBoundingBoxDf.shape[0] * training_data_percent)
training_patientIds_box = trainLabelAndBoundingBoxDf['patientId'][0:training_data_end_index]
validation_patientIds_box = trainLabelAndBoundingBoxDf['patientId'][training_data_end_index:]

params = {'dim': (256, 256), 'batch_size': 32, 'shuffle': True}
training_data_generator_box = LungDataGenerator(training_patientIds_box, **params)
validation_data_generator_box = LungDataGenerator(validation_patientIds_box, **params)

history_unet = model_unet.fit(training_data_generator_box, validation_data=validation_data_generator_box, callbacks=[learning_rate], epochs=epoch_count, workers=4, use_multiprocessing=True)

In [ ]:
print('Below is the representation of variation of loss, accuracy and mean_iou for training and validation data.')
plt.figure(figsize=(15,6))
plt.subplot(121)
plt.plot(history_unet.epoch, history_unet.history["loss"], label="Train loss")
plt.plot(history_unet.epoch, history_unet.history["val_loss"], label="Valid loss")
plt.legend()
plt.subplot(122)
plt.plot(history_unet.epoch, history_unet.history["mean_iou"], label="Train iou")
plt.plot(history_unet.epoch, history_unet.history["val_mean_iou"], label="Valid iou")
plt.legend()
plt.show()

In [ ]:
print('We can now analyze the confusion matrix and classification report for U-Net model.')
cm_unet = get_confusion_matrix(model_unet, validation_data_generator_box, True)

In [ ]:
print('We can now analyze the predictions on the validation data.')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
for imgs, masks, targets in validation_data_generator_box:
    # Predict a batch of images
    predictions = model_unet.predict(imgs)
    # Create a plot for the images
    fig, axes = plt.subplots(3, 5, figsize=(20, 15))
    axes = axes.ravel()
    axidx = 0
    
    # Loop through the batch
    for img, mask, prediction, target in zip(imgs, masks, predictions, targets):
        axes[axidx].imshow(img[:,:,0])
        
        # Apply threshold to true mask
        comp = mask[:,:,0] > 0.5
        
        # Apply connected components.
        comp = measure.label(comp)
        
        # Apply bounding boxes
        for region in measure.regionprops(comp):
            y1, x1, y2, x2 = region.bbox
            height = y2-y1
            width = x2-x1
            axes[axidx].add_patch(patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='b', facecolor='none'))
            
        # Apply threshold to the predicted mask
        comp = prediction[:,:,0] > 0.5
        
        # Apply connected components.
        comp = measure.label(comp)
        
        predTarget = 0
        
        # Apply bounding boxes
        for region in measure.regionprops(comp):
            # If a region is found mark the target value as 1
            predTarget = 1
            y1, x1, y2, x2 = region.bbox
            height = y2-y1
            width = x2-x1
            axes[axidx].add_patch(patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='r', facecolor='none'))
        
        axes[axidx].text(0.05, 0.95, f'Actual target: {target} \nPredicted target: {predTarget}', transform=axes[axidx].transAxes, fontsize=14, verticalalignment='top', bbox=props)
        axidx += 1
        if(axidx >= 15):
            break;
        
    plt.show()
    
    # Display only one plot per batch
    break


In [ ]:
print('Now we can compare the results of the Res-Net and U-Net architectures used.')
print('First we can compare the changes in the loss and mean_iou for validation data for ResNet vs U-Net architectures')
plt.figure(figsize=(15,6))
plt.subplot(121)
plt.plot(history.epoch, history.history["val_loss"], label="Res-Net Validation loss")
plt.plot(history.epoch, history_unet.history["val_loss"], label="U-Net Validation loss")
plt.legend()
plt.subplot(122)
plt.plot(history.epoch, history.history["val_mean_iou"], label="Res-Net Validation IOU")
plt.plot(history.epoch, history_unet.history["val_mean_iou"], label="U-Net Validation IOU")
plt.legend()
plt.show()

print('Now we can compare the confusion matrices for Res-Net and U-Net architectures.')
def annotate_plot(plot):
    for p in plot.patches:
        plot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')  

cm_comparison_df = pd.DataFrame(np.array([['Res-Net', cm[0][0], cm[0, 1], cm[1,0], cm[1,1]], 
                                          ['U-Net', cm_unet[0][0], cm_unet[0, 1], cm_unet[1,0], cm_unet[1,1]]])
                               , columns=['architecture', 'True_Negatives', 'False_Positives', 'False_Negatives', 'True_Positives'])


cm_comparison_df['architecture'] = cm_comparison_df['architecture'].astype('category')

fig, axes = plt.subplots(2, 2)
true_negative_plot = sns.barplot(x='architecture', y='True_Negatives', data=cm_comparison_df, ax=axes[0][0])
false_positive_plot = sns.barplot(x='architecture', y='False_Positives', data=cm_comparison_df, ax=axes[0][1])
false_negative_plot = sns.barplot(x='architecture', y='False_Negatives', data=cm_comparison_df, ax=axes[1][0])
true_positive_plot = sns.barplot(x='architecture', y='True_Positives', data=cm_comparison_df, ax=axes[1][1])
annotate_plot(true_negative_plot)
annotate_plot(false_positive_plot)
annotate_plot(false_negative_plot)
annotate_plot(true_positive_plot)
fig.set_figwidth(15)
fig.set_figheight(12)
fig.suptitle('Confusion matrix comparison for Res-Net and U-Net architectures')
fig.tight_layout(pad=5.0)
plt.show()

In [ ]:
print('We can see that the overall performance for the Res-Net architecture is better than the U-Net architecture. One point to \
note is that the number of false negatives is greater in case of Res-Net. It is important to lower the number of false negatives \
as we may end up calling out a patient with pneumonia as not having issues, however, as the target classification is done based \
on the presence of bounding boxes, we can attempt to improve the IOU metric for ResNet in order to reduce the number of false \
negatives.')
print('1) Changing the number of channels.')
params = {'dim': (256, 256), 'batch_size': 32, 'shuffle': True}
training_data_generator_box = LungDataGenerator(training_patientIds_box, **params)
validation_data_generator_box = LungDataGenerator(validation_patientIds_box, **params)

model_channel = create_network(input_size=256, channels=16, n_blocks=2, depth=4, isDropoutAdded=False)

model_channel.compile(optimizer='adam', loss=iou_bce_loss, metrics=['accuracy', mean_iou])
history_channel = model_channel.fit(training_data_generator_box, validation_data=validation_data_generator_box, callbacks=[learning_rate], epochs=epoch_count, workers=4, use_multiprocessing=True)
cm_channel = get_confusion_matrix(model_channel, validation_data_generator_box, False)

In [ ]:
print('2) Changing the number of blocks.')
params = {'dim': (256, 256), 'batch_size': 32, 'shuffle': True}
training_data_generator_box = LungDataGenerator(training_patientIds_box, **params)
validation_data_generator_box = LungDataGenerator(validation_patientIds_box, **params)
model_blocks = create_network(input_size=256, channels=32, n_blocks=4, depth=4, isDropoutAdded=False)

model_blocks.compile(optimizer='adam', loss=iou_bce_loss, metrics=['accuracy', mean_iou])
history_blocks = model_blocks.fit(training_data_generator_box, validation_data=validation_data_generator_box, callbacks=[learning_rate], epochs=epoch_count, workers=4, use_multiprocessing=True)
cm_blocks = get_confusion_matrix(model_blocks, validation_data_generator_box, False)

In [ ]:
print('3) Changing the depth of the model.')
params = {'dim': (256, 256), 'batch_size': 32, 'shuffle': True}
training_data_generator_box = LungDataGenerator(training_patientIds_box, **params)
validation_data_generator_box = LungDataGenerator(validation_patientIds_box, **params)
model_depth = create_network(input_size=256, channels=32, n_blocks=2, depth=6, isDropoutAdded=False)

model_depth.compile(optimizer='adam', loss=iou_bce_loss, metrics=['accuracy', mean_iou])
history_depth = model_depth.fit(training_data_generator_box, validation_data=validation_data_generator_box, callbacks=[learning_rate], epochs=epoch_count, workers=4, use_multiprocessing=True)
cm_depth = get_confusion_matrix(model_depth, validation_data_generator_box, False)

In [ ]:
print('4) Changing the dimension of the image that is given as input.')
params = {'dim': (512, 512), 'batch_size': 16, 'shuffle': True}
training_data_generator_box = LungDataGenerator(training_patientIds_box, **params)
validation_data_generator_box = LungDataGenerator(validation_patientIds_box, **params)
model_dim = create_network(input_size=512, channels=32, n_blocks=2, depth=4, isDropoutAdded=False)

model_dim.compile(optimizer='adam', loss=iou_bce_loss, metrics=['accuracy', mean_iou])
history_dim = model_dim.fit(training_data_generator_box, validation_data=validation_data_generator_box, callbacks=[learning_rate], epochs=epoch_count, workers=4, use_multiprocessing=True)
cm_dim = get_confusion_matrix(model_dim, validation_data_generator_box, False)

In [ ]:
print('5) Adding dropout layers to ResNet based model.')
params = {'dim': (256, 256), 'batch_size': 32, 'shuffle': True}
training_data_generator_box = LungDataGenerator(training_patientIds_box, **params)
validation_data_generator_box = LungDataGenerator(validation_patientIds_box, **params)
model_dropout = create_network(input_size=256, channels=32, n_blocks=2, depth=4, isDropoutAdded=True)

model_dropout.compile(optimizer='adam', loss=iou_bce_loss, metrics=['accuracy', mean_iou])
history_dropout = model_dropout.fit(training_data_generator_box, validation_data=validation_data_generator_box, callbacks=[learning_rate], epochs=epoch_count, workers=4, use_multiprocessing=True)
cm_dropout = get_confusion_matrix(model_dropout, validation_data_generator_box, False)

In [ ]:
print('6) Changing the learning rate to 0.01.')
params = {'dim': (256, 256), 'batch_size': 32, 'shuffle': True}
training_data_generator_box = LungDataGenerator(training_patientIds_box, **params)
validation_data_generator_box = LungDataGenerator(validation_patientIds_box, **params)
model_lr = create_network(input_size=256, channels=32, n_blocks=2, depth=4, isDropoutAdded=False)

learn_rate = 0.01
adamOpt01 = keras.optimizers.Adam(learning_rate=learn_rate)
model_lr.compile(optimizer=adamOpt01, loss=iou_bce_loss, metrics=['accuracy', mean_iou])

# Define cosine annealing
def cosine_annealing01(x):
    lr = learn_rate
    epochs = epoch_count
    return lr * (np.cos(np.pi * x / epochs) + 1.) / 2

learning_rate01 = tf.keras.callbacks.LearningRateScheduler(cosine_annealing01)
history_lr = model_lr.fit(training_data_generator_box, validation_data=validation_data_generator_box, callbacks=[learning_rate01], epochs=epoch_count, workers=4, use_multiprocessing=True)
cm_lr = get_confusion_matrix(model_lr, validation_data_generator_box, False)

In [ ]:
targetModel = 'Res-Net'
print(f'Now we can compare the results of the variations of the {targetModel} architecture.')
print(f'First we can compare the changes in the loss and mean_iou for validation data for different {targetModel} architectures')
plt.figure(figsize=(15,6))
plt.subplot(121)
plt.plot(history_channel.epoch, history_channel.history["val_loss"], label="Validation loss Post channel tuning")
plt.plot(history_blocks.epoch, history_blocks.history["val_loss"], label="Validation loss Post number of blocks tuning")
plt.plot(history_depth.epoch, history_depth.history["val_loss"], label="Validation loss Post depth tuning")
plt.plot(history_dim.epoch, history_dim.history["val_loss"], label="Validation loss Post dimension tuning")
plt.plot(history_dropout.epoch, history_dropout.history["val_loss"], label="Validation loss Post adding dropouts")
plt.plot(history_lr.epoch, history_lr.history["val_loss"], label="Validation loss Post changing learning rate")
plt.legend()
plt.subplot(122)
plt.plot(history_channel.epoch, history_channel.history["val_mean_iou"], label="Validation IOU Post channel tuning")
plt.plot(history_blocks.epoch, history_blocks.history["val_mean_iou"], label="Validation IOU Post number of blocks tuning")
plt.plot(history_depth.epoch, history_depth.history["val_mean_iou"], label="Validation IOU Post depth tuning")
plt.plot(history_dim.epoch, history_dim.history["val_mean_iou"], label="Validation IOU Post dimension tuning")
plt.plot(history_dropout.epoch, history_dropout.history["val_mean_iou"], label="Validation IOU Post adding dropouts")
plt.plot(history_lr.epoch, history_lr.history["val_mean_iou"], label="Validation IOU Post changing learning rate")
plt.legend()
plt.show()

print(f'Now we can compare the confusion matrices for validation data for different {targetModel} architectures.')
def annotate_plot(plot):
    for p in plot.patches:
        plot.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')  

def get_cm_array(cmObj):
    return cmObj[0][0], cmObj[0, 1], cmObj[1,0], cmObj[1,1]

get_cm_array(cm_channel)

cm_comparison_df = pd.DataFrame(np.array([[f'{targetModel}_channel', cm_channel[0][0], cm_channel[0, 1], cm_channel[1,0], cm_channel[1,1]], 
                                          [f'{targetModel}_blocks', cm_blocks[0][0], cm_blocks[0, 1], cm_blocks[1,0], cm_blocks[1,1]],
                                          [f'{targetModel}_depth', cm_depth[0][0], cm_depth[0, 1], cm_depth[1,0], cm_depth[1,1]],
                                          [f'{targetModel}_dim', cm_dim[0][0], cm_dim[0, 1], cm_dim[1,0], cm_dim[1,1]],
                                          [f'{targetModel}_dropouts', cm_dropout[0][0], cm_dropout[0, 1], cm_dropout[1,0], cm_dropout[1,1]],
                                          [f'{targetModel}_learningrate', cm_lr[0][0], cm_lr[0, 1], cm_lr[1,0], cm_lr[1,1]]
                                         ])
                               , columns=['architecture', 'True_Negatives', 'False_Positives', 'False_Negatives', 'True_Positives'])


cm_comparison_df['architecture'] = cm_comparison_df['architecture'].astype('category')

fig, axes = plt.subplots(2, 2)
true_negative_plot = sns.barplot(x='architecture', y='True_Negatives', data=cm_comparison_df, ax=axes[0][0])
false_positive_plot = sns.barplot(x='architecture', y='False_Positives', data=cm_comparison_df, ax=axes[0][1])
false_negative_plot = sns.barplot(x='architecture', y='False_Negatives', data=cm_comparison_df, ax=axes[1][0])
true_positive_plot = sns.barplot(x='architecture', y='True_Positives', data=cm_comparison_df, ax=axes[1][1])
annotate_plot(true_negative_plot)
annotate_plot(false_positive_plot)
annotate_plot(false_negative_plot)
annotate_plot(true_positive_plot)
axes[0,0].tick_params('x', labelrotation=45)
axes[0,1].tick_params('x', labelrotation=45)
axes[1,0].tick_params('x', labelrotation=45)
axes[1,1].tick_params('x', labelrotation=45)
fig.set_figwidth(15)
fig.set_figheight(12)
fig.suptitle('Confusion matrix comparison for varitions of Res-Net.')
fig.tight_layout(pad=5.0)
plt.show()

## Conclusion

Based on the metrics for validation loss, mean IoU and confusion matrix comparison, the **Resnet model** with modified dimension of image as **512\*512** has the least number of **false negatives (approx. 150)** and has a **low validation loss of around 0.05**. Thereby, we can consider the resnet model variation having the initial image size as 512\*512 as the target model for the peneumonia detection model.

***Below are few interesting observations:***
1. We were able to identify the bounding boxes from the mask by segmenting the image with the help of **skimage.measure (regionprops) method**.
2. On **increasing the depth** of the Res-Net architecture, the number of **false negatives increased drastically** leading us to avoid the increase in depth of the model.

***Below are few challenges we faced during model generation and tuning:***
1. As the images were heavy (1024X1024), we had to **reduce the size of the image** in order to cope up with the restricted memory availability.
2. The model training was taking excessive time because of which we made use of **GPU acceleration** to speed up the process.8

***Further model optimization plan:***
1. We can try combining the **variations in the hyperparameters (number of blocks/channel/depth, dimension, dropouts, learning rate)** to get a better mean IoU and loss. 
2. Additionally, we can utilize **image augmentation** on the class of images with pneumonia to balance the entire data set. This can help improve the model.
3. For further optimizations of the selected model for real-life usage, we can train the model on images from the problem domain using **transfer learning** (making the last few layers of the model learnable).
